In [1]:
import collections
import heapq

# -----------------------------
# 1. Huffman coding
# -----------------------------
class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq

def huffman_encode(text):
    freq = collections.Counter(text)
    heap = [HuffmanNode(ch, f) for ch, f in freq.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        merged = HuffmanNode(None, n1.freq + n2.freq)
        merged.left, merged.right = n1, n2
        heapq.heappush(heap, merged)

    codes = {}
    def generate_codes(node, current=""):
        if node is None:
            return
        if node.char is not None:
            codes[node.char] = current
        generate_codes(node.left, current + "0")
        generate_codes(node.right, current + "1")
    generate_codes(heap[0])

    encoded = "".join(codes[ch] for ch in text)
    return encoded, codes

def huffman_encode_words(text):
    # Tokeniser par mots
    words = text.split()  # Par défaut, séparation sur les espaces

    # Calculer la fréquence des mots
    freq = collections.Counter(words)

    # Créer les feuilles de l'arbre de Huffman
    heap = [HuffmanNode(w, f) for w, f in freq.items()]
    heapq.heapify(heap)

    # Construire l'arbre de Huffman
    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        merged = HuffmanNode(None, n1.freq + n2.freq)
        merged.left, merged.right = n1, n2
        heapq.heappush(heap, merged)

    # Générer les codes binaires
    codes = {}
    def generate_codes(node, current=""):
        if node is None:
            return
        if node.char is not None:
            codes[node.char] = current
        generate_codes(node.left, current + "0")
        generate_codes(node.right, current + "1")

    root = heap[0]
    generate_codes(root)

    # Encoder le texte
    encoded = " ".join(codes[word] for word in words)
    return encoded, codes, root  # Retourne aussi l’arbre pour décodage

def huffman_decode_words(encoded, root):
    decoded_words = []
    current = root
    for bit in encoded.split():
        node = root
        for b in bit:
            node = node.left if b == '0' else node.right
        decoded_words.append(node.char)
    return ' '.join(decoded_words)

# -----------------------------
# 2. Byte Pair Encoding (BPE)
# -----------------------------
def bpe_encode(text, num_merges=50):
    text = list(text)
    for _ in range(num_merges):
        # count pairs
        pairs = collections.Counter(zip(text, text[1:]))
        if not pairs:
            break
        best_pair = max(pairs, key=pairs.get)
        new_symbol = "".join(best_pair)

        # replace pair
        new_text = []
        i = 0
        while i < len(text):
            if i < len(text) - 1 and (text[i], text[i+1]) == best_pair:
                new_text.append(new_symbol)
                i += 2
            else:
                new_text.append(text[i])
                i += 1
        text = new_text
    return text

# -----------------------------
# 3. LZW compression
# -----------------------------
def lzw_encode(text):
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}

    w = ""
    result = []
    for c in text:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c
    if w:
        result.append(dictionary[w])
    return result


In [2]:
def lz77_encode(text, window_size=400, lookahead_buffer_size=15):
    i = 0
    output = []

    while i < len(text):
        # Fenêtre glissante : zone dans le texte déjà vue
        start_window = max(0, i - window_size)
        window = text[start_window:i]

        # Zone de recherche dans la fenêtre
        match_length = 0
        match_distance = 0
        # Chercher la plus longue correspondance dans la fenêtre
        for j in range(len(window)):
            length = 0
            while (length < lookahead_buffer_size and
                   i + length < len(text) and
                   window[j + length] == text[i + length]):
                length += 1
                if j + length >= len(window):
                    break
            if length > match_length:
                match_length = length
                match_distance = len(window) - j

        if match_length >= 3:
            # On encode la séquence répétée
            next_char = text[i + match_length] if i + match_length < len(text) else ''
            output.append((match_distance, match_length, next_char))
            i += match_length + 1
        else:
            # Pas de correspondance significative : on encode juste le caractère brut
            output.append((0, 0, text[i]))
            i += 1

    return output
def estimate_lz77_size(encoded_triplets, bits_offset=9, bits_length=4, bits_char=8):
    bits_per_triplet = bits_offset + bits_length + bits_char
    total_bits = len(encoded_triplets) * bits_per_triplet
    total_bytes = (total_bits + 7) // 8  # round up to nearest byte
    return total_bytes

In [3]:
with open('data/text_code/code1.py') as f:
    text = f.read()
print("Taille originale :", len(text), "caractères")
huff_encoded, huff_codes = huffman_encode(text)
print("Huffman :", len(huff_encoded) // 8, "octets environ")

bpe_encoded = bpe_encode(text, num_merges=100)
print("BPE :", len(bpe_encoded), "symboles (vs", len(text), "caractères)")

lzw_encoded = lzw_encode(text)
print("LZW :", len(lzw_encoded), "entiers")

encoded, codes, root = huffman_encode_words(text)
encoded_length = len(encoded)
print("Huffman (mots) :", encoded_length, "octets estimés")

lz77encoded = lz77_encode(text)
lz77_size = estimate_lz77_size(lz77encoded)
print(f"LZ77 : {len(lz77encoded)} triplets, taille estimée : {lz77_size} octets")

Taille originale : 4264 caractères
Huffman : 2488 octets environ
BPE : 1679 symboles (vs 4264 caractères)
LZW : 1529 entiers
Huffman (mots) : 3693 octets estimés
LZ77 : 1284 triplets, taille estimée : 3371 octets


In [4]:
with open('data/text_natural/text_natural_1.txt') as f:
    text = f.read()
print("Taille originale :", len(text), "caractères")
huff_encoded, huff_codes = huffman_encode(text)
print("Huffman :", len(huff_encoded) // 8, "octets environ")

bpe_encoded = bpe_encode(text, num_merges=100)
print("BPE :", len(bpe_encoded), "symboles (vs", len(text), "caractères)")

#lzw_encoded = lzw_encode(text)
#print("LZW :", len(lzw_encoded), "entiers")

Taille originale : 2830 caractères
Huffman : 1552 octets environ
BPE : 1398 symboles (vs 2830 caractères)


In [5]:
with open('data/text_structured/accumulation-accounts-2008-2023-provisional.csv') as f:
    text = f.read()
print("Taille originale :", len(text), "caractères")
huff_encoded, huff_codes = huffman_encode(text)
print("Huffman :", len(huff_encoded) // 8, "octets environ")

#bpe_encoded = bpe_encode(text, num_merges=100)
#print("BPE :", len(bpe_encoded), "symboles (vs", len(text), "caractères)")

lzw_encoded = lzw_encode(text)
print("LZW :", len(lzw_encoded), "entiers")

lz77encoded = lz77_encode(text)
lz77_size = estimate_lz77_size(lz77encoded)
print(f"LZ77 : {len(lz77encoded)} triplets, taille estimée : {lz77_size} octets")

Taille originale : 4297756 caractères
Huffman : 2595917 octets environ
LZW : 194363 entiers


KeyboardInterrupt: 